In [83]:
import cv2
import math
from win32api import GetSystemMetrics
import time
import numpy as np
import os
import imutils

In [84]:
# Image path
imagePath = 'Images'
imageName = "sample3.PNG"
imagePath = os.path.join(imagePath,imageName)
#road detection works better with images where the scale is smaller per unit pixel 
#(screen distance = lesser real world distance)
#It also works better with a different style of image

mountingPoints = []
quadPoints = []

Steps to do test/implement ray masking
1. Plot out viewing quadrilateral
2. Plot out mounting point
3. Mask out for building contours
4. Find corners of building contours

Taking the nearest corner, find the slope of the line joining the mounting point and this nearest corner. Find the intersection point of this line and the farthest edge of the quadrilateral (What if farthest edge is out of bounds ??)

In [85]:
#using new bgr values for the new image in low red.
def getBorderContour_text(img):

    # Upper and lower color limit customized for snazzy maps
    low_red = (55, 55, 255)
    high_yellow = (242,251,256)

    low_gray = (241,241,241)
    high_gray = (244,243,241)

    # create masks
    red_mask = cv2.inRange(img, low_red, low_red )
    
    # combine masks
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(red_mask, cv2.MORPH_DILATE,kernel)

    blank = np.zeros(img.shape[:2], dtype='uint8')

    masked = cv2.bitwise_and(img,img,mask=combined_mask)

    # findcontours
    cnts=cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        # cv2.drawContours(img, [c], -1, (255,0,255), thickness=1)
        area = cv2.contourArea(c)
        if(area>200):
            for eps in np.linspace(0.001, 0.0075, 10):
                # approximate the contour
                peri = cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, eps * peri, True)
            
            # draw the approximated contour on the image  
            cv2.drawContours(blank, [approx], -1, 255, thickness=1)
            # cv2.drawContours(img, [c], -1, 255, thickness=1)


    # cv2.imshow("image",img)
    # cv2.waitKey(0)

    return blank,masked

In [86]:
def click_MountingnScale_points(event, x, y, flags, params):

    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
        mountingPoints.append((x,y))

        # displaying the coordinates on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(x) + ',' +str(y), (x,y), font,1, (255, 0, 0), 2)
        cv2.imshow('img', img)


    # checking for right mouse clicks    
    if event==cv2.EVENT_RBUTTONDOWN:
        quadPoints.append((x,y))

        # displaying the coordinates on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(x) + ',' +str(y),(x,y), font, 1,(0, 0, 255), 2)
        cv2.imshow('img', img)

In [87]:
def getCorners(masked):

    operatedImage = cv2.cvtColor(masked, cv2.COLOR_BGR2GRAY)

    # modify the data type
    # setting to 32-bit floating point
    # operatedImage = cv2.fastNlMeansDenoisingColored(operatedImage,None,10,10,7,21) #uncomment if you feel image is noisy (not needed)
    operatedImage = np.float32(operatedImage)

    # apply the cv2.cornerHarris method
    # to detect the corners with appropriate
    # values as input parameters
    #~~~~~~~~~~~~~#
    #for obtaining corners for ray masking
    #~~~~~~~~~~~~~#
    dest = cv2.cornerHarris(operatedImage, 2, 5, 0.07)

    # Results are marked through the dilated corners
    dest = cv2.dilate(dest, None)

    # for pts in poly_pts:
    #     dest[pts[0]][pts[1]] = 0
    # Reverting back to the original image,
    # with optimal threshold value
    size = masked.shape

    suro = np.zeros(size, dtype='uint8')
    suro[dest > 0.01 * dest.max()]=[255, 255, 255]

    blank = np.zeros(masked.shape[:2], dtype='uint8')
    poly_pts = np.array( quadPoints ,dtype=np.int32)
    poly_pts = poly_pts.reshape((-1, 1, 2))
    polymask = cv2.fillPoly(blank, pts=[poly_pts],color=255)

    suro = cv2.bitwise_and(suro,suro,mask=polymask)

    counter = 0
    for y in range(len(suro)):
        for x in range(len(suro[y])):
            if(suro[y,x,0]==255 and suro[y,x,1]==255 and suro[y,x,2]==255):
                counter = counter + 1
    print(f"in house counter = {counter}")

    cv2.imshow("suro",suro)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    #     # De-allocate any associated memory 
    # cv2.waitKey(0)

    return suro


In [88]:
def findClusterCenters(img):

    corner_centers = []

    blank = np.zeros(img.shape, dtype='uint8')
    # blank_copy = blank.copy()

    # create masks
    white_mask = cv2.inRange(img, (255,255,255), (255,255,255))

    # combine masks
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(white_mask, cv2.MORPH_DILATE,kernel)


    cnts=cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for i in cnts:
        M = cv2.moments(i)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
        print(f"center - {cx},{cy}")
        corner_centers.append([cx,cy])
        cv2.circle(blank, (cx, cy), 1, (255, 255, 255), -1)
        # cv2.drawContours(blank, [i], -1, 0, -1)
        # corner_centers.append([{cx},{cy}])
    cv2.imshow("points", blank)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return corner_centers,blank




In [89]:
img = cv2.imread(imagePath)
cv2.imshow("img",img)
cv2.setMouseCallback('img', click_MountingnScale_points)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [90]:
img = cv2.imread(imagePath)
bldg_brdrs, bldg_mask = getBorderContour_text(img)
cv2.imshow("bldng_brdrs",bldg_brdrs)
cv2.imshow("bldng_mask",bldg_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [91]:
highlighted_corners = getCorners(bldg_mask)
cornerPoints,point_corners = findClusterCenters(highlighted_corners)
highlighted_corners = cv2.bitwise_or(img, highlighted_corners)
point_corners = cv2.bitwise_or(img, point_corners)
cv2.imshow('Image with Corners', highlighted_corners)
cv2.imshow('Image with Point Corners', point_corners)
cv2.waitKey(0)
cv2.destroyAllWindows()

counter = 0
for y in range(len(highlighted_corners)):
    for x in range(len(highlighted_corners[y])):
        if(highlighted_corners[y,x,0]==255 and highlighted_corners[y,x,1]==255 and highlighted_corners[y,x,2]==255):
            counter = counter + 1
print(counter)

in house counter = 98
center - 296,304
center - 415,301
center - 376,286
center - 324,284
center - 289,196
center - 472,140
center - 471,77
center - 300,75
98


In [92]:
mountingPoint = mountingPoints[0]

slope_farthest_edge = (quadPoints[4][1]-quadPoints[3][1])/(quadPoints[4][0]-quadPoints[3][0]) #slope of farthest edge line

c2 = quadPoints[4][1] - slope_farthest_edge*quadPoints[4][0] #constant of farthest edge line
for cornerPoint in cornerPoints:
    slope = (mountingPoint[1] - cornerPoint[1])/(mountingPoint[0] - cornerPoint[0])
    #find the intersection point b/w the farthest edge line and the line joining cornerPoint and mountingPoint
    #points are selected such that quadPoints[3] and quadPoints[4] represent the points for the farthest edge
    

    #finding intersection point of both lines associated w/ slope
    #x_intercept = (c2-c1)/(m1-m2)
    #y_intercept = m1*x_intercept + c1

    c1 = mountingPoint[1] - slope*mountingPoint[0] #constant of line joining mounting point and corner = y1 - mx1

    x_intercept = (c2-c1)/(slope - slope_farthest_edge)

    y_intercept = slope*x_intercept + c1

    #slope_cut_off is slope of line joining corner point and intercept point on farthest edge
    slope_
